In [ ]:
from planet4.io import get_image_id_data as get_image_id, get_image_name_data,\
    get_list_of_image_names_data
from planet4 import io

In [ ]:
from planet4.region_data import Inca

In [ ]:
data = get_list_of_image_names_data(Inca.season1)

In [ ]:
data.info()

In [ ]:
image_ids = data.image_id.unique()

In [ ]:
df = data[data.image_id==image_ids[0]]

In [ ]:
df.info()

In [ ]:
df.groupby(['classification_id','user_name']).size()

In [ ]:
def process_image_name(image_name):

    def process_user_group(g):
        c_id = g.sort('created_at').classification_id.iloc[0]
        return g[g.classification_id == c_id]

    data = df[df.image_name == image_name]
    data = data.groupby(['user_name']).apply(
        process_user_group).reset_index(drop=True)
    fname = 'temp_' + image_name + '.h5'
    data.to_hdf(fname, 'df')


In [ ]:
def process_user_group(g):
    c_id = g.sort('created_at').classification_id.iloc[0]
    return g[g.classification_id == c_id]

In [ ]:
filtered_data = df.groupby(['user_name']).apply(process_user_group)

In [ ]:
from planet4 import helper_functions as hf

In [ ]:
def process_image_id(image_id):
    df = data[data.image_id==image_id]
    n_class = df.classification_id.unique().size
    results = hf.classification_counts_per_user(df).value_counts()
    if not any(results.index>1):
        return n_class, n_class
    else:
        n_class_real = results[1]
        for index in results.index[results.index>1]:
            n_class_real += results[index]
        return (n_class_real, n_class)

In [ ]:
real_class_percents = []
for image_id in image_ids:
    real_class_percents.append(process_image_id(image_id))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
s = pd.DataFrame(real_class_percents, index=image_ids, columns=['real_n_class', 'expected_n_class'])

In [ ]:
s.head()

In [ ]:
s.plot()

In [ ]:
import seaborn as sns
sns.set_context('talk')

In [ ]:
s.head()

In [ ]:
s = s.assign(fraction=s.real_n_class/s.expected_n_class)

In [ ]:
s.fraction.plot(style='.',title='Inca City, season1, fraction of good classifications')

In [ ]:
s.describe()

In [ ]:
df = data[data.image_id=='APF0000zea']
n_class= df.classification_id.unique().size
n_class
results = hf.classification_counts_per_user(df).value_counts()
results
results.index>1
n_class_real = results[1]
for index in results.index[results.index>1]:
    n_class_real += results[index]
n_class_real

# Update

In [1]:
from planet4 import io

In [2]:
from planet4 import reduction

In [3]:
db = io.DBManager('/Users/klay6683/data/planet4/2015-10-11_planet_four_classifications_queryable2.h5')

In [4]:
data = db.get_image_id_markings('APF00003hp')

In [35]:
pd.HDFStore(db.dbname, 'df', where='image_name')

'/Users/klay6683/data/planet4/2015-10-11_planet_four_classifications_queryable2.h5'

In [5]:
from planet4 import helper_functions as hf

In [7]:
hf.classification_counts_per_user(data)

user_name
Kitharode                                         2
bmacpherson                                       2
not-logged-in-073dee28bbc9c250d9dc02cb99f4ef93    2
GazyB                                             2
stephen milanovic                                 1
tonnie1                                           1
not-logged-in-a73525bd89cfa4ce77d75d2fe53fcb2d    1
CosmoMallard                                      1
ewa@gorska.co.uk                                  1
simonwalker                                       1
garydougill                                       1
Hannah93                                          1
D_J_G                                             1
LivingMars                                        1
pauly1080                                         1
Juliandmark                                       1
timtrent                                          1
ray149                                            1
not-logged-in-a2fe0761f89efa289864177a0d14dcc3    1
ke

In [9]:
def pug1(g):
    c_id = g.sort_values(by='classification_id').classification_id.iloc[0]
    return g[g.classification_id == c_id]

In [10]:
def pug1b(g):
    c_id = g.sort_values(by='classification_id').classification_id.iloc[0]
    return c_id

In [11]:
def pug2(g):
    c_id = g.classification_id.min()
    return g[g.classification_id == c_id]

In [12]:
def pug2b(g):
    c_id = g.classification_id.min()
    return c_id

In [21]:
usergroup = data.groupby(['user_name'], sort=False)

In [22]:
%timeit usergroup.apply(pug1).reset_index(drop=True)

10 loops, best of 3: 169 ms per loop


In [23]:
%timeit usergroup.apply(pug2).reset_index(drop=True)

10 loops, best of 3: 132 ms per loop


In [25]:
%timeit data[data.classification_id.isin(usergroup.classification_id.min())]

100 loops, best of 3: 6.95 ms per loop


In [26]:
v1 = usergroup.apply(pug1).reset_index(drop=True).sort_values(by=['classification_id'])

In [27]:
v2 = usergroup.apply(pug2).reset_index(drop=True).sort_values(by=['classification_id'])

In [31]:
v3 = data[data.classification_id.isin(usergroup.classification_id.min())]

In [32]:
(v1.dropna() == v3.dropna()).all()

classification_id    True
created_at           True
image_id             True
image_name           True
image_url            True
user_name            True
marking              True
x_tile               True
y_tile               True
acquisition_date     True
local_mars_time      True
x                    True
y                    True
image_x              True
image_y              True
radius_1             True
radius_2             True
distance             True
angle                True
spread               True
version              True
dtype: bool

In [34]:
v3[v3.classification_id=='50ef44b995e6e42d8c000001']

,classification_id,created_at,image_id,image_name,image_url,user_name,marking,x_tile,y_tile,acquisition_date,...,x,y,image_x,image_y,radius_1,radius_2,distance,angle,spread,version


In [30]:
(usergroup.apply(pug1).dropna() == usergroup.apply(pug2).dropna()).all()

classification_id    True
created_at           True
image_id             True
image_name           True
image_url            True
user_name            True
marking              True
x_tile               True
y_tile               True
acquisition_date     True
local_mars_time      True
x                    True
y                    True
image_x              True
image_y              True
radius_1             True
radius_2             True
distance             True
angle                True
spread               True
version              True
dtype: bool

In [ ]:
v1.info()

In [ ]:
(v1.dropna()==v2.dropna()).all()

In [ ]:
v2.head()

In [ ]:
data.groupby(['user_name','classification_id']).apply(lambda x: len(x.classification_id.unique())).sort_values(ascending=False).min()

In [ ]:
v3 = data[data.classification_id.isin(data.groupby('user_name').classification_id.max())].sort_values(by='classification_id')

In [ ]:
(v1.classification_id.sort_values() == v2.classification_id.sort_values()).all()

In [ ]:
(g.apply(pug1).reset_index(drop=True) == g.apply(pug2).reset_index(drop=True)).all()

In [ ]:
%timeit g.apply(pug2b)

In [ ]:
img_ids = ['APF00003hp']
users = ['not-logged-in-073dee28bbc9c250d9dc02cb99f4ef93']

for img_id, user in zip(img_ids, users):
    print("image_id: ", img_id)
    print("User: ", user)
    data = db.get_image_id_markings(img_id)
    print("Before filtering classification_id created_at times:")
    print(data[data.user_name==user].created_at.unique())
    print("Classification_ids:")
    print(data[data.user_name==user].classification_id.unique())

    g = data.groupby(['user_name'])
    res = g.apply(process_user_group).reset_index(drop=True)
    print("After filtering:")
    print(res[res.user_name==user].created_at.unique())
    print(res[res.user_name==user].classification_id.unique())
    print()

In [ ]:
fname = '/Users/klay6683/data/planet4/2015-10-11_planet_four_classifications_queryable.h5'
db = io.DBManager()

In [ ]:
db.dbname

In [ ]:
data = pd.read_hdf(db.dbname, 'df',
                    where="classification_id=='50ef44b795e6e42cd2000001'")
data

In [ ]:
df = pd.read_hdf(db.dbname, 'df')

In [ ]:
df[df.classification_id=='50ef44b795e6e42cd2000001']

In [ ]:
df[df.classification_id=='50ef44b995e6e42d8c000001']

In [ ]:
df[df.classification_id=='50ee0e5694b9d564a90000b5']

In [ ]:
db.dbname

In [ ]:
df.classification_id = df.classification_id.astype('str')

In [ ]:
df.to_hdf('/Users/klay6683/data/planet4/2015-10-11_planet_four_classifications_queryable_cleaned.h5',
         'df', format='t', data_columns=reduction.data_columns)

In [ ]:
pd.read_hdf('/Users/klay6683/data/planet4/2015-10-11_planet_four_classifications_queryable_cleaned.h5',
           'df', where="classification_id=='50ef44b795e6e42cd2000001'")

In [ ]:
fname = '/Users/klay6683/data/planet4/2015-10-11_planet_four_classifications_queryable.h5'

In [ ]:
reduction.remove_duplicates_from_file(fname)

In [ ]:
data = pd.read_hdf('testing.h5', 'df',
                    where="classification_id=='50ef44b995e6e42d8c000001'")
data

In [ ]:
data2 = db.get_class_id_data('50ef44b995e6e42d8c000001')

In [36]:
s = pd.Series(list('abc'))

In [37]:
pd.DataFrame(s)

,0
0,a
1,b
2,c


In [1]:
from planet4 import io

In [4]:
db=io.DBManager()

In [3]:
import time

In [5]:
imgnames = db.season2and3_image_names

In [6]:
where = "image_name in {}".format(imgnames.values.tolist())

In [7]:
where

"image_name in ['ESP_011544_0985', 'ESP_021684_0985', 'ESP_011697_0980', 'ESP_020322_0930', 'ESP_021455_0935', 'ESP_020214_0935', 'ESP_012008_0975', 'ESP_020930_0980', 'ESP_020357_0950', 'ESP_012884_0935', 'ESP_021497_0980', 'ESP_011900_0985', 'ESP_012291_0980', 'ESP_021494_0945', 'ESP_021454_0925', 'ESP_021522_0930', 'ESP_020339_0985', 'ESP_012604_0965', 'ESP_012254_1065', 'ESP_021605_0985', 'ESP_011350_0945', 'ESP_020376_0980', 'ESP_021526_0985', 'ESP_021460_0985', 'ESP_012076_0945', 'ESP_012344_0950', 'ESP_011729_0985', 'ESP_012063_0945', 'ESP_012212_0950', 'ESP_012256_0985', 'ESP_020598_0935', 'ESP_011565_0930', 'ESP_012693_0950', 'ESP_012838_0950', 'ESP_020888_0935', 'ESP_011737_0980', 'ESP_020677_0980', 'ESP_020115_0985', 'ESP_011447_0950', 'ESP_012436_0980', 'ESP_021671_0985', 'ESP_012211_0930', 'ESP_020783_0950', 'ESP_012744_0985', 'ESP_020282_0930', 'ESP_012691_0985', 'ESP_020049_0985', 'ESP_021491_0950', 'ESP_021520_0925', 'ESP_020571_0945', 'ESP_020731_0930', 'ESP_022273_095

In [9]:
import time
t0 = time.time()
season23 = pd.read_hdf(db.dbname, 'df', where=where)
t1 = time.time()
print("time: ", t1 - t0)

time:  19.6526939868927
